# Объявление методов

Объявление метода напоминает объявление функции, но дополнительно появляется ещё один параметр перед названием функции. Этот параметр связывает метод с типом.

In [1]:
import "math"

type Point struct {
    X float64
    Y float64
}

// Обычная функция.
func Distance(p, q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

// Метод для типа Point.
func (p Point) Distance(q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

Дополнительный параметр p называют **получателем**(receiver) метода, наследство от первых ООП языков, в которых вызов метода назывался "отправкой сообщения объекту".

В Go не принято называть получатель специальным именем типа this или self. Он называется также как и обычный параметр функции. Т.к. имя получателя будет использоваться довольно часто, желательно подобрать ему короткое имя и использовать именно его во всех методах. Обычно выбирают **первую букву** типа как p для Point в нашем примере.

Для вызова метода сначала указывают получатель, а затем через точку метод:

In [2]:
import "fmt"
p1 := Point{1, 2}
p2 := Point{4, 6}
d := p1.Distance(p2) // Вызов метода.
fmt.Sprint(d)

5


Между объявлением функции Distance и методом Distance типа Point нет никакого конфликта: функция Distance является функцией уровня пакета, а метод принадлежит типу Point.

Выражение p.Distance называется **селектором**(selector), т.к. он выбирает метод Distance получателя p. Селекторы также используются для выбора полей, как например, p.X. Т.к. методы и поля типа находятся в одном и том же пространстве имён, нельзя давать им одинаковые имена.

Т.к. каждый тип определяет своё собственное пространство имён, можно объявлить метод Distance и для другого типа:

In [3]:
type Path []Point

func (p Path) Distance() float64 {
    sum := 0.0
    for i := range p {
        if i > 0 {
            sum += p[i-1].Distance(p[i])    
        }
    }
    return sum
}

Path является именованным типом, а не структурой как в случае с Point, но всё равно для него тоже можно определить метод. Иногда бывает удобно определить новые методы для простых типов как числовой, строковый и т.д., т.к. мы  получаем функционал этих базовых типов бесплатно. Методы можно объявить для любого именованного типа в этом же пакете, кроме типов, для которых базовым типом является указатель или интерфейс.

## Получатель-указатель

Т.к. при передаче аргумента в функции создаётся его копия, для возможности изменения самой переменной необходимо передать в функцию её адрес. То же самое касается и методов, которым нужно обновить получатель: эти методы необходимо связывать с **указателем** на тип:

In [4]:
import "math"

type Point struct {
    X float64
    Y float64
}

// Метод для типа Point.
func (p Point) Distance(q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

// Метод с получателем указателем.
func (p *Point) ScaleBy(factor float64) {
    p.X *= factor
    p.Y *= factor
}

В более реалистичных программах, если хотя бы один метод типа требует получатель-указатель, то и **все** методы этого типа должны быть объявлены с получателем-указателем, даже если им это не нужно. В нашем примере мы нарушили это правило, чтобы продемонстрировать работу обоих типов методов.

Именованные типы(Point) и указатели на них(\*Point) являются единственными типами, которые могут использоваться при объявлении получателя. **Нельзя** объявить метод для типа, который сам по себе является указателем:

In [5]:
type P *int

func (p P) foo() {}

3:7: invalid receiver P (pointer or interface type)


Метод ScaleBy можно вызвать несколькими способами, предоставив \*Point получатель:

In [6]:
// 1
r := &Point{0, 3}
r.ScaleBy(2)
fmt.Println(*r)

// 2
p1 := Point{4, 3}
ptr := &p1
ptr.ScaleBy(2)
fmt.Println(*ptr)

// 3
p2 := Point{-2, 0}
(&p2).ScaleBy(2)
fmt.Sprint(p2)

{0 6}
{8 6}
{-4 0}


Последние два способа являются довольно неуклюжими. К счастью здесь нам может помочь язык. Если получатель p **является переменной** типа Point, но метод требует получатель \*Point, то мы можем просто вызвать этот метод как обычный:

In [7]:
p := Point{4, 2}
p.ScaleBy(0.5)
fmt.Sprint(p)

{2 1}


Компилятор сам неявно выполнит операцию получения адреса переменной &p. Это работает и для полей структуры и элементов массива, но для временных значений нет:

In [8]:
Point{0, 0}.ScaleBy(1)

1:1: invalid operation: ScaleBy is not in method set of github.com/yunabe/lgo/sess7b2274696d65223a313533343331313339363437323234363833337d/exec4.Point


Для получателей указателей можно вызывать обычные методы как Distance:

In [9]:
ptr := &Point{9, 12}
d := ptr.Distance(Point{9, 12})
fmt.Sprint(d)

0


Давайте подведём итог всему сказанному, т.к. вызов методов часто вызывает путаницу. В каждом допустимом выражении вызова метода верно только одно из следующих утверждений:  
**1.** И аргумент и получатель имеют одинаковый тип T или \*T:

In [ ]:
Point{0,0}.Distance(Point{3,3}) // Point  
ptr.ScaleBy(3)                  // *Point

**2.** Аргумент имеет тип T, а получатель \*T. Компилятор неявно получает адрес переменной:

In [ ]:
p.ScaleBy(2)

**3.** Аргумент имеет тип \*T, а получатель T. Компилятор неявно разыменовывает аргумент:

In [ ]:
ptr.Distance(Point{0,0})

Если все методы именованного типа T имеют получатель типа T(не \*T), то его можно свободно копировать; вызов любого метода всегда создаёт копию. Но если хотя бы один метод имеет получатель-указатель, то мы должны избегать копирования объектов типа T.

## Передача nil получателю

Если получатель метода является указателем, то этот метод можно вызывать и для nil значения этого типа. Особенно это имеет смысл, когда nil является осмысленным нулевым значением для типа. Например, для списка целых чисел nil может означать пустой список:

In [10]:
// Список целых чисел.
// nil *IntList представляет собой пустой список.
type IntList struct {
    Value int
    Tail  *IntList
}

func (l *IntList) Sum() int {
    if l == nil {
        return 0
    }
    return l.Value + l.Tail.Sum()
}

var ls IntList
fmt.Println(ls.Tail == nil)
fmt.Sprint(ls.Sum())

true
0


При объявлении типа, получатели методов которого могут принимать nil значения, лучше явно указать об этом в документирующих комментариях, как мы это сделали для примера списка целых чисел.